In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
import time
import pandas as pd
import os
import urllib.request
import PIL
import requests
from PIL import Image
from instabot import Bot
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import json
import warnings
warnings.simplefilter('ignore')
from instagrapi import Client
from instagrapi.story import StoryBuilder, StoryMention
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

In [ ]:
def load_images(link,path):
    return urllib.request.urlretrieve(link,f"{path}\\{'_'.join(link.split('/')[10:12])}.jpg")

In [ ]:
def get_data(file_path):
    result=[]
    df=pd.read_excel(file_path)    
    folder_name=datetime.today().strftime('%Y-%m-%d')
    parent_dir = r"C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Saat"  #save path must be changed
    path = os.path.join(parent_dir, folder_name)
    os.makedirs(path)
    driver = webdriver.Chrome(executable_path=r'C:\Users\svnduw\.wdm\drivers\chromedriver\win32\99.0.4844.51\\chromedriver.exe')
    for url in df['urls']:        
        driver.get(url)        
        soup=BeautifulSoup(driver.page_source,'lxml')
        
        try:
            brand=soup.find('h1',class_='pr-new-br').find('a').text.strip()
        except:
            brand=None
        try:
            saler=soup.find('div',class_='merchant-box-wrapper').find('a').text.strip()
        except:
            saler=None
        try:
            saler_link=f"https://www.trendyol.com{soup.find('div',class_='merchant-box-wrapper').find('a').get('href')}"
        except:
            saler_link=None
        try:
            prices=soup.find('div',class_='product-price-container').find('div',class_='pr-bx-w').find_all('span')
            price_list=[]
            image_links=[]
            for price in prices:
                price_list.append(float(price.text.strip().split()[0].replace(',','.')))
        except:
            price_list=None
        images=soup.find_all('div',class_='product-slide')
        if len(images)!=0: 
            image_folder=str('_'.join(soup.find_all('div',class_='product-slide')[1].find('img').get('src').split('/')[13:14]))
            for image in images:
                image_links.append('/'.join(image.find('img').get('src').split('/')[:3]+image.find('img').get('src').split('/')[6:]))
        else:
            image_folder=str('_'.join(soup.find('div',class_='product-container').find('div').find('img').get('src').split('/')[13:14]))
            image_links.append('/'.join(soup.find('div',class_='product-container').find('div').find('img').get('src').split('/')[:3]+soup.find('div',class_='product-container').find('div').find('img').get('src').split('/')[6:]))       
        for image_link in image_links:
            load_images(image_link,path)
        result.append({
            'brand':brand,
            'saler':saler,
            'saler_link':saler_link,
            'price':price_list,
            'image_name':image_folder,
            'image_links':image_links,
            'page_link':url
        })    
    df=pd.json_normalize(result)
    df['get_price']=df['price'].apply(lambda x: max(x))
    lira=get_lira('https://www.xe.com/currencyconverter/convert/?Amount=1&From=TRY&To=KZT')
    df['lira_to_kzt']=lira
    df['delivery']=6000   #delivery from Turkey to Kazakhstan
    df['price_insta']=(df['get_price']*df['lira_to_kzt']*1.4+df['delivery']).round(-3).astype(int)
    df['profit']=df['price_insta']-df['get_price']*df['lira_to_kzt']-df['delivery']
    file_name=datetime.today().strftime('%Y-%m-%d')
    df.to_excel(f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\Saat\\отчеты\\{file_name}.xlsx',index=False)
    return 'Done'

In [ ]:
def get_lira(link):
    response=requests.get(link)
    soup=BeautifulSoup(response.text,'lxml')
    lira=int(round(float(soup.find('p',class_='result__BigRate-sc-1bsijpp-1 iGrAod').text.strip().split()[0]),0))
    return lira

In [ ]:
def resize(image_pil):
    image_pil = (Image.open(image_pil, 'r')) 
    width = 1080
    height = 1080

    ratio_w = width / image_pil.width
    ratio_h = height / image_pil.height
    if ratio_w < ratio_h:        
        resize_width = width
        resize_height = round(ratio_w * image_pil.height)
    else:        
        resize_width = round(ratio_h * image_pil.width)
        resize_height = height
    image_resize = image_pil.resize((resize_width, resize_height), Image.ANTIALIAS)
    background = Image.new('RGBA', (width, height), (255, 255, 255, 255))
    offset = (round((width - resize_width) / 2), round((height - resize_height) / 2))
    background.paste(image_resize, offset)
    return background.convert('RGB')

In [ ]:
def check_url_from_old(file_path,link_file_path):
    files=os.listdir(file_path)
    df=pd.DataFrame()
    for file in files:
        temp=pd.read_excel(f'{file_path}//{file}')
        df=df.append(temp,ignore_index=True)
        new_urls=pd.read_excel(f'{link_file_path}')
        new_urls=new_urls[~new_urls['urls'].isin(df.page_link.tolist())]
        save_path='\\'.join(link_file_path.split('\\')[:-1])
        new_urls.to_excel(f"{save_path}\\links.xlsx",index=False)

In [ ]:
check_url_from_old(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Saat\отчеты',r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Saat\links.xlsx')

In [ ]:
get_data(r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Saat\links.xlsx')

In [ ]:
folder_name=datetime.today().strftime('%Y-%m-%d')
parent_dir = r"C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Saat"  #save path must be changed
new_path = os.path.join(parent_dir, folder_name,'resized')
os.makedirs(new_path)
old_path='/'.join(new_path.split('\\')[:-1])
files=os.listdir(old_path)
for file in files:
    if file.endswith('jpg'):
        resize(old_path+'\\'+file).save(new_path+'\\'+file)

In [ ]:
def post_photos(file_path,image_path):
    df=pd.read_excel(file_path)
    files=os.listdir(image_path)
    Username='qyran_watches'
    Password='****'
    bot = Client()
    bot.login(Username, Password)
    for index,row in df.iterrows():
        new=[]        
        for file in files:
            if str(row['image_name'])==str(file.split('_')[0]):
                new.append(f'{image_path}\\{file}') 
        if len(new)!=1:
            text = '⌚ '+str(row['brand'])+'\n 🔴 Производство: Турция \n 💵 Цена: '+str(row['price_insta'])+'\n ♻️ Для заказа можете написать нам по прямой ссылке в шапке профиля \n 📲 8 707 946 6496 \n 💸Предоплата 50% \n_____________________ \n Доставка по всему РК\n_ _ _ _ _ _ _ _ _ _ _ _ _\n Наши магазины:\n @ant_wears - интернет-магазин женской (мусульманской) одежды\n @gstyle_lady - интернет-магазин женской одежды\n @social_shoe_store - интернет-магазин обуви\n @aishoes.kz - интернет-магазин женской обуви\n @ayalin_kids - интернет-магазин детской одежды\n @ayamin_bags - интернет-магазин люксовых сумок'
            
            bot.album_upload(
            new,
            caption = text)
        else:
            text = '⌚ '+str(row['brand'])+'\n 🔴 Производство: Турция \n 💵 Цена: '+str(row['price_insta'])+'\n ♻️ Для заказа можете написать нам по прямой ссылке в шапке профиля \n 📲 8 707 946 6496 \n 💸Предоплата 50% \n_____________________ \n Доставка по всему РК\n_ _ _ _ _ _ _ _ _ _ _ _ _\n Наши магазины:\n @ant_wears - интернет-магазин женской (мусульманской) одежды\n @gstyle_lady - интернет-магазин женской одежды\n @social_shoe_store - интернет-магазин обуви\n @aishoes.kz - интернет-магазин женской обуви\n @ayalin_kids - интернет-магазин детской одежды\n @ayamin_bags - интернет-магазин люксовых сумок'
            
            bot.photo_upload(new[0],caption=text)

In [ ]:
data=datetime.today().strftime('%Y-%m-%d')
post_photos(f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\Saat\\отчеты\\{data}.xlsx',f'C:/Users/svnduw/Desktop/Bekbol/Bekbol project/Saat/{data}/resized')

In [ ]:
def post_stories(image_path):
    Username='qyran_watches'
    Password='*****'
    bot = Client()
    bot.login(Username, Password)
    files = os.listdir(image_path)
    adw0rd = bot.user_info_by_username('qyran_watches')
    for file in files:
         bot.photo_upload_to_story(image_path+'\\'+file,
                             "@qyran_watches",
                             mentions=[StoryMention(user=adw0rd, x=0.49892962, y=0.703125, width=0.8333333333333334, height=0.125)])    

In [ ]:
post_stories(f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\Saat\\{data}\\resized')
